# A test of Variational Monte Carlo on some Toy Models

Here we only deal with a toy model based on a Jastrow solution of Fermi-Hubbard model. We first validate this method in 1D, then apply this to the 2D case. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

## Fermi-Hubbard model

$$ H = -t \sum_{\langle ij \rangle} \sum_{\sigma} c_{i, \sigma}^{\dagger} c_{j, \sigma} + U \sum_{i} n_{i\uparrow} n_{i\downarrow}$$

(For 2D, there are more N.N. sites)

periodic boundary condition

## Noninteracting state (choose U=0 here instead of Hartree-Fock)

$$ c_{i\uparrow} = d_i $$

$$ c_{i\downarrow} = d_{i+L} $$

index $I$ running from 1 to 2L


In [96]:
# system size (1D)
L = 4
N = 2
U = 2         # take tunneling strength as a unit
H1 = np.zeros((2*L,2*L))
for i in range(2*L-1):
    H1[i, i+1] = -1
H1[L-1, 0] = -1
H1[2*L-1, L] = -1
H1[L-1, L] = 0
H = H1 + H1.T
e, phi = np.linalg.eig(H)
phi_0 = phi[: , np.argsort(e)[:N]]
# print(e, np.argsort(e), phi_0)

## Jastrow-like Wavefunction

$$ |\psi \rangle = exp( -v \sum_{i} n_{i\uparrow} n_{i\downarrow}) |\phi _0 \rangle $$

In [81]:
# initialization
v = np.random.randn()
x = np.random.choice(2*L, N, replace=False)
site_empty = np.delete(np.arange(2*L), x)
x_ext = x - L
site_double = np.intersect1d(x, x_ext)
site_single = np.setdiff1d(x, site_double)
site_double_number = np.size(site_double)
print(x, x_ext, site_empty, site_double, site_double_number)


[1 6] [-3  2] [0 2 3 4 5 7] [] 0


## Computation of expectations and variances of local operators $O$ (here for Hamiltonian $H$ especially)

$$ \langle O \rangle = \frac{\langle \Psi_J |O| \Psi_J \rangle}{\langle \Psi_J | \Psi_J \rangle} = \frac{ \sum_x |\langle \Psi_J| x\rangle |^2 \frac{\langle x |O| \Psi_J \rangle}{\langle x| \Psi_J \rangle}}{\sum_x |\langle \Psi_J| x\rangle |^2} = \sum_x p_J(x) O_L(x) $$

$$ H_{local} = \frac{\langle x |H| \Psi_J \rangle}{\langle x| \Psi_J \rangle} = \frac{ \sum_{x^{\prime}} \langle x |T| x^{\prime} \rangle \Psi_J(x^{\prime}) + U(x)\Psi_J(x)}{\Psi_J(x)} $$

$$ H^2_{local} = \frac{\langle x |(U+T)^2| \Psi_J \rangle}{\langle x| \Psi_J \rangle}  = \frac{ \sum_{x^{\prime \prime}} \langle x |T^2| x^{\prime \prime} \rangle \langle x^{\prime \prime}| \Psi_J \rangle + U(x) \langle x |T| \Psi_J \rangle + \sum_{x^{\prime}}\langle x |T| x^{\prime} \rangle U(x^{\prime}) \Psi_J(x^{\prime})+ U^2(x)\Psi_J(x)}{\ \Psi_J(x)} $$

$| x^{\prime} \rangle $ and $| x^{\prime \prime} \rangle $ are configurations differ from $| x \rangle $ with only one and two hopping and no change in spin.

So there are three terms about $| x^{\prime} \rangle $ and one term about $| x^{\prime \prime} \rangle $


In [103]:
# energy and variance
E = 0
sigma = 0

psi_jx = np.linalg.det(phi_0[x])

def onehop():
    pass

def twohop():
    pass

# 由于T不改变自旋，故要分成两个子空间，并且注意边界条件！
# 顺序重要吗？


## Update of determinant $|\frac{\phi_0(x^\prime)}{\phi_0(x)}|^2$ :

$x^\prime$ differs from x with only one index, x ranges from 0 to 2L-1 (spin up for 0 ~ L-1) 

### Some details: 

Notation:



## Update of Jastrow factor:

Since there are only diagnoal terms in Gutzwiller factor, we only need to keep track with doubly occupied sites, which are nonvanishing in the sum on the exponential term.


In [79]:
# randomly choose a new configuration
index_change = np.random.randint(N)
index_new = np.random.randint(2*L-1-N)
x_new = x.copy()
x_new[index_change] = site_empty[index_new]
# print(x, site_double, index_change, index_new, x_new)

# update of Jastrow factor
site_double_number_new = site_double_number

if x[index_change]%L in site_double:
    arg_delete = np.argwhere(site_double, x[index_change]%L)[0]
    site_double_new = np.delete(site_double, site_double[arg_delete])
    site_double_number_new += (-1)

if x_new[index_change]%L in site_single:
    arg_append = np.argwhere(site_double, x[index_change]%L)[0]
    site_double_new = np.delete(site_double, site_double[arg_delete])
    site_double_number_new += (-1)


# update of determinant

def detUpdate(delta_v,delta_index,U_inv,U_det,axis):  # detUpdate result could be minus det, but we don't care here                                   
    if axis == 0:
        detRatio = 1 + np.dot(U_inv[:,delta_index],delta_v)
    elif axis == 1:
        detRatio = 1 + np.dot(U_inv[delta_index,:],delta_v) # delta_v input can be either row or column vector
    return detRatio * U_det     # return value is a complex number
    
# following update methods dealing with enlarged matrix with row and collum added at the end
# update the inv and det of enlarged matrix with O(n^2)
def matrixDetUpdate(m_new,m_det,m_inv,n): # m_new is n*n, m is (n-1)*(n-1)
    v_new = m_new[0:-1,-1]
    delta_v_mat = np.array([v_new,]*(n-1)).T - m_new[0:-1,0:-1]
    m_det_list = np.array([detUpdate(delta_v_mat[:,i],i,m_inv,m_det,1) for i in range(n-1)] + [(-1 ** (n-1))*m_det])     
    # only care about relative +-
    m_new_det = np.dot(m_new[-1],m_det_list)
    return m_new_det

def matrixInvUpdate(m_new,m_inv,n): # m_new is n*n, m is (n-1)*(n-1)
    u = np.expand_dims(m_new[0:-1,-1],axis=1)
    v = np.expand_dims(m_new[-1,0:-1],axis=0)
    c = m_new[-1,-1]
    k_inv = 1 / (c - v.dot(m_inv).dot(u))
    m_new_inv = m_inv + k_inv * np.dot(m_inv.dot(u),v.dot(m_inv))
    m_new_inv = np.c_[m_new_inv,-k_inv * m_inv.dot(u)]
    m_new_inv = np.r_[m_new_inv,np.expand_dims(np.append(-k_inv * v.dot(m_inv), np.array([k_inv])),axis=0)]
    return m_new_inv


# Recept in Markov chain
r = 0.2
if np.random.rand() < np.min(1, r):
    pass



[1 6]
[1 6] [] 0 1 [2 6]


In [ ]:
# energy



In [ ]:
# variation


## Place for simple test

In [87]:
# test
a = np.arange(2,5)
np.argwhere(3)

array([[0]])